In [ ]:
# Includes
from labeyrieClasses import target, deconvolved
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Import binary FITS data, calc PSD
binary = target()
binary.fits.fileName = "/home/niels/Documents/FITS/KP330.fits"
binary.fits.read(numDimensions = 3)
binary.psdCalc()

# Import reference FITS data, calc PSD
reference = target()
reference.fits.fileName = "/home/niels/Documents/FITS/KP331.fits"
reference.fits.read(numDimensions = 3)
reference.psdCalc()

In [ ]:
# Display reference and binary PSDs
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(np.log10(binary.psd.data))
plt.title("Binary PSD")
plt.subplot(1,2,2)
plt.imshow(np.log10(reference.psd.data))
plt.title("Reference PSD")

# Wiener Filter:
$ \mbox{Estimated Original Image} = \hat{F}(u,v)$ 

$ \mbox{Original Image} = F(u,v)$

$ \mbox{Noise PSD} = S_{n}(u,v)$

$ \mbox{Original Image PSD} = S_{f}(u,v)$

$ \hat{F}(u,v) = [\frac{H(u,v)}{H^{2}(u,v)+S_{n}(u,v)/S_{f}(u,v)}]G(u,v) $

$ S_{n}(u,v)/S_{f}(u,v) = SNR $

For white noise, $S_{n}(u,V) = k = constant $

$ \hat{F}(u,v) = [\frac{H(u,v)}{H^{2}(u,v)+k/S_{f}(u,v)}]G(u,v) $


Some approximations use $S_{n}(u,v)/S_{f}(u,v) = l = constant$

In [ ]:
# Setting up Wiener Filter Variables
G = binary.psd.data
H = reference.psd.data


In [ ]:
# Estimating original image with SNR = constant assumption
l = 1E-3

F_hat = (H/(H**2+l))*G
F_hat_acorr = np.fft.fftshift(np.real(np.fft.ifft2(np.fft.fftshift(F_hat))))

plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.imshow(np.log10(F_hat))
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(F_hat_acorr)
plt.colorbar()